# Fine-tuning GPT-4.1 to Write LinkedIn Posts (in my style)
## ABB #7 - Session 5

Code authored by: Shaw Talebi

### imports

In [1]:
import pandas as pd
import json
import random

import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# import sk from .env file
load_dotenv()

# connect to openai API
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### functions

In [3]:
def clean_post(text):
    # Split into lines
    lines = text.split('\n')
    # Remove leading/trailing quotes and whitespace, and filter out empty lines
    cleaned_lines = [line.strip().strip('"') for line in lines]
    # Join back into a single string
    return '\n'.join(cleaned_lines)

## Step 1: Input-output Pairs

### Load Data

In [4]:
# read data
df = pd.read_csv('data/LI_posts.csv')

In [5]:
# change column names
df.columns = ['date', 'link', 'post', 'idea']

In [6]:
# Set dtypes
df = df.astype({
    'date': str,
    'link': str,
    'post': str,
    'idea': str
})

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

In [7]:
# set index
df = df.set_index('date')

### Data Prep

In [8]:
# pre-process posts
df['post'] = df['post'].apply(clean_post)

In [9]:
# replace idea with first line of post
df['idea'] = df['post'].str.split('\n').str[0]

In [10]:
df.shape

(669, 3)

In [11]:
# drop rows with posts less than 3 lines
df = df[df['post'].str.split('\n').str.len() >= 3]

In [12]:
df.shape

(638, 3)

## Step 2: Prompt Template

### Create training examples

In [13]:
# construct training examples
example_list = []

system_prompt = """# LinkedIn Ghostwriter

You are a LinkedIn Ghostwriter for Shaw Talebi, an AI educator and entrepreneur.

Given a post idea's first line from the user, generate a post in Shaw's unique style.

Include the following in each post:
- A compelling opening 1-2 lines that hooks the reader
- Copy that expands upon the idea in valuable way
- A call to action or share relevant content
- Don't use hashtags
"""

for i in range(len(df)):    
    system_dict = {"role": "system", "content": system_prompt}
    user_dict = {"role": "user", "content": df['idea'].iloc[i]}
    assistant_dict = {"role": "assistant", "content": df['post'].iloc[i]}
    
    messages_list = [system_dict, user_dict, assistant_dict]
    
    example_list.append({"messages": messages_list})

In [14]:
print(example_list[0]['messages'][0]['content'])
print("---")
print(example_list[0]['messages'][1]['content'])
print("---")
print(example_list[0]['messages'][2]['content'])

# LinkedIn Ghostwriter

You are a LinkedIn Ghostwriter for Shaw Talebi, an AI educator and entrepreneur.

Given a post idea's first line from the user, generate a post in Shaw's unique style.

Include the following in each post:
- A compelling opening 1-2 lines that hooks the reader
- Copy that expands upon the idea in valuable way
- A call to action or share relevant content
- Don't use hashtags

---
LLM capabilities are doubling every 7 months…
---
LLM capabilities are doubling every 7 months…

Here’s the most important LLM benchmark I’ve come across 👇 

A couple of months ago, the team at METR released a new AI benchmark.

Rather than evaluating AI systems in terms of accuracy on well-known datasets or artificial tasks, it evaluates them on real-world tasks measured in average human task completion time.

In other words, they took 170 tasks, measured how long it typically takes a human to do each, then evaluated whether an AI system could do each with >50% accuracy.

Current models 

In [15]:
len(example_list)

638

## Step 3: Create train/validation split

In [16]:
# randomly pick out validation examples
num_examples = 68
validation_index_list = random.sample(range(0, len(example_list)-1), num_examples)
validation_data_list = [example_list[index] for index in validation_index_list]

for example in validation_data_list:
    example_list.remove(example)

In [17]:
print(len(example_list))
print(len(validation_data_list))

570
68


In [18]:
# write examples to file
with open('data/train-data.jsonl', 'w') as train_file:
    for example in example_list:
        json.dump(example, train_file)
        train_file.write('\n')

with open('data/valid-data.jsonl', 'w') as valid_file:
    for example in validation_data_list:
        json.dump(example, valid_file)
        valid_file.write('\n')

### Upload data to OpenAI

In [19]:
train_file = client.files.create(
  file = open("data/train-data.jsonl", "rb"),
  purpose = "fine-tune"
)

valid_file = client.files.create(
  file = open("data/valid-data.jsonl", "rb"),
  purpose = "fine-tune"
)

## Step 4: Fine-tune model

In [20]:
client.fine_tuning.jobs.create(
    training_file = train_file.id,
    validation_file = valid_file.id,
    suffix = "LI-post-writer",
    model = "gpt-4.1-mini-2025-04-14",
    method={
    "type": "supervised",
    "supervised": {
      "hyperparameters": {
        "n_epochs": 3,
        "learning_rate_multiplier": 1.25,
        "batch_size": 1,
            }
        }
    }
)

FineTuningJob(id='ftjob-f18vbmVebAdhTwi1RUFEjhs6', created_at=1763077224, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.25, n_epochs=3), model='gpt-4.1-mini-2025-04-14', object='fine_tuning.job', organization_id='org-KjWERyZ9WLUqIdrdMeJh4zC0', result_files=[], seed=372366926, status='validating_files', trained_tokens=None, training_file='file-7crdcax2n2CgHmLbJwi5py', validation_file='file-BMxFq3xhT9LiWeBgDhx1c2', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.25, n_epochs=3))), user_provided_suffix='LI-post-writer', usage_metrics=None, shared_with_openai=False, eval_id=None)

## Step 5: Evaluate fine-tuned model

In [21]:
def generate_post(system_prompt, model_name, idea):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": idea}
        ],
        temperature=0.7,
    )
    return response.choices[0].message.content

In [22]:
idea = "5 things I learned from 5 years on YouTube"

#### GPT-4.1-mini (no fine-tuning)

In [23]:
model_name = "gpt-4.1-mini-2025-04-14"
# model_name = "gpt-4.1-2025-04-14"

# read (long) system prompt
with open("prompts/linkedin-prompt.md", "r") as file:
    system_prompt_long = file.read()

print(generate_post(system_prompt_long, model_name, idea))

Step 1: Purpose and Audience  
- Purpose: Share practical lessons learned from five years of creating content on YouTube.  
- Audience: Content creators, entrepreneurs, and anyone interested in building an online presence.

Step 2: Wireframe  
- Hook: A clear, curiosity-driven statement about lessons from five years on YouTube.  
- Body: List five specific, practical things learned, each with a brief explanation.  
- CTA: Ask readers to share their own practical insights or tools for content creation.

Step 3: Body (Meat)  
- 1) Consistency in publishing builds audience over time.  
- 2) Content quality improves with practice and feedback.  
- 3) Engaging with your community helps sustain growth.  
- 4) Analytics guide you to what works and what doesn’t.  
- 5) Experimenting with formats and styles is essential to find your voice.

Step 4: CTA  
- What practical lessons have you learned from creating content over time?

Step 5: Hook Directions  
- [User’s own phrasing] “5 things I lear

#### GPT-4.1-mini (fine-tuned)

In [24]:
model_name = "ft:gpt-4.1-mini-2025-04-14:shawhin-talebi-ventures-llc:li-post-writer:CaR7nerw"

# print(system_prompt, "\n--")
print(generate_post(system_prompt, model_name, idea))

5 things I learned from 5 years on YouTube

1) Use a "hook" - get to the point quickly and don't waste people's time
2) Use visuals - I can't explain something in 30 seconds using words alone
3) Repurpose content - I've turned 1 YouTube video into 10 pieces of content
4) Content > Camera equipment - I can easily outproduce any competition with a better idea
5) Don't fight the algorithm - post more and post consistently 

You can see how I apply these lessons in my latest video: https://lnkd.in/gM6qG2tV


In [25]:
# # delete files (after fine-tuning is done)
# client.files.delete(train_file.id)
# client.files.delete(valid_file.id)